In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # clear shared RAM (Colab shares RAM and GPU with other user sessions, this guarantees all resources allocated to this session)
# # memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil --quiet
# !pip install psutil --quiet
# !pip install humanize --quiet
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # only one GPU on Colab and isn't guaranteed
# gpu = GPUs[0]
# proc = psutil.Process(os.getpid())
# print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " | Proc size: " + humanize.naturalsize(proc.memory_info().rss))
# print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

In [3]:
import numpy as np
import pandas as pd
import torch
import torchvision as tv
import re
import time
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import Module, GRU, Embedding, Linear, Sigmoid, CrossEntropyLoss, ReLU, Tanh, Sequential
from torch import nn
from torchvision import transforms
import torch.optim as optim
from torchvision.utils import make_grid, save_image
from tqdm import tqdm
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.datasets as datasets
import imageio
import numpy as np
import matplotlib
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from tqdm import tqdm
import copy
matplotlib.style.use('ggplot')

In [4]:
# learning parameters
batch_size = 512
epochs = 200
sample_size = 64 # fixed sample size
nz = 128 # latent vector size
k = 1 # number of steps to apply to the discriminator
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,)),
])
to_pil_image = transforms.ToPILImage()

In [6]:
fmnist = datasets.FashionMNIST(root='./', train=True, download=True, transform=transform)
data_loader = DataLoader(fmnist, batch_size=batch_size, shuffle=True)

In [7]:
class Generator(Module):
    def __init__(self, nz):
        super(Generator, self).__init__()
        self.nz = nz
        self.main = Sequential(
            Linear(self.nz, 256),
            ReLU(),

            Linear(256, 512),
            ReLU(),

            Linear(512, 784),
            Tanh(),
        )

    def forward(self, x):
        return self.main(x).view(-1, 1, 28, 28)


In [8]:
class Discriminator(Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.n_input = 784
        self.main = Sequential(
            Linear(self.n_input, 1024),
            ReLU(),
            Linear(1024, 512),
            ReLU(),
            Linear(512, 1),
            nn.Sigmoid(),
        )
    def forward(self, x):
        x = x.view(-1, 784)
        return self.main(x)

Initialize the Neural Networks and Define the Optimizers

In [9]:


generator = Generator(nz).to(device)
discriminator = Discriminator().to(device)
# unrolled_discriminator = Discriminator().to(device)
print('##### GENERATOR #####')
print(generator)
print('######################')
print('\n##### DISCRIMINATOR #####')
print(discriminator)
print('######################')
# print('\n##### UNROLLED DISCRIMINATOR #####')
# print(unrolled_discriminator)
# print('######################')

##### GENERATOR #####
Generator(
  (main): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=784, bias=True)
    (5): Tanh()
  )
)
######################

##### DISCRIMINATOR #####
Discriminator(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)
######################


In [10]:
# optimizers

optim_g = optim.Adam(generator.parameters(), lr=0.0002)
optim_d = optim.Adam(discriminator.parameters(), lr=0.0002)
optim_ud = optim.Adam(discriminator.parameters(), lr=0.0002)

In [11]:
# loss function
criterion = nn.BCELoss()


In [12]:
losses_g = [] # to store generator loss after each epoch
losses_d = [] # to store discriminator loss after each epoch
lossed_ud = []
images = [] # to store images generatd by the generator

In [13]:
# to create real labels (1s)
def label_real(size):
    data = torch.ones(size, 1)
    return data.to(device)
# to create fake labels (0s)
def label_fake(size):
    data = torch.zeros(size, 1)
    return data.to(device)

In [14]:
def create_noise(sample_size, nz):
    return torch.randn(sample_size, nz).to(device)

In [15]:
# to save the images generated by the generator
def save_generator_image(image, path):
    save_image(image, path)

In [16]:
# function to train the discriminator network
def train_discriminator(optimizer, data_real, data_fake, discriminator):
    b_size = data_real.size(0)
    real_label = label_real(b_size)
    fake_label = label_fake(b_size)
    optimizer.zero_grad()
    output_real = discriminator(data_real)
    loss_real = criterion(output_real, real_label)
    output_fake = discriminator(data_fake)
    loss_fake = criterion(output_fake, fake_label)
    loss_real.backward()
    loss_fake.backward()
    optimizer.step()
    return loss_real + loss_fake

In [17]:
# function to train the generator network
def train_generator(optimizer, data_fake, discriminator):
    b_size = data_fake.size(0)
    real_label = label_real(b_size)
    optimizer.zero_grad()
    output = discriminator(data_fake)
    loss = criterion(output, real_label)
    loss.backward()
    optimizer.step()
    return loss

In [18]:
# create the noise vector
noise = create_noise(sample_size, nz)
generator.train()
discriminator.train()

Discriminator(
  (main): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [19]:
# path = "/content/drive/MyDrive/Deep_Learning/HW3/outputs_unrolled_gan/"
path = "./models/unrolled/"
epochs = 200
# k = 10

losses_g = [] # to store generator loss after each epoch
losses_d = [] # to store discriminator loss after each epoch
losses_ud = []
images = [] # to store images generatd by the generator
length = 0.

for epoch in range(epochs):
    start = time.time()
    # length = 0.
    loss_g = 0.0
    loss_d = 0.0
    loss_ud = 0.
    for bi, data in enumerate(data_loader):
        image, _ = data
        image = image.to(device)
        b_size = len(image)
        # run the discriminator for k number of steps
        for step in range(k):
            # print(create_noise(b_size, nz).shape)
            data_fake = generator(create_noise(b_size, nz)).detach()
            data_real = image
            # train the discriminator network
            loss_d += train_discriminator(optim_d, data_real, data_fake, discriminator)

            # unroll
            unrolled_discriminator = copy.deepcopy(discriminator)
            # print(unrolled_discriminator)
            loss_ud += train_discriminator(optim_ud, data_real, data_fake, unrolled_discriminator)
            

        data_fake = generator(create_noise(b_size, nz))
        # train the generator network
        loss_g += train_generator(optim_g, data_fake, unrolled_discriminator)
        del unrolled_discriminator
    # create the final fake image for the epoch
    generated_img = generator(noise).cpu().detach()
    # make the images as grid
    generated_img = make_grid(generated_img)
    # save the generated torch tensor models to disk
    save_generator_image(generated_img, path + f"gen_img{epoch}.png")
    images.append(generated_img)
    epoch_loss_g = loss_g / bi # total generator loss for the epoch
    epoch_loss_d = loss_d / bi # total discriminator loss for the epoch
    epoch_loss_ud = loss_ud / bi
    losses_g.append(epoch_loss_g)
    losses_d.append(epoch_loss_d)
    losses_ud.append(epoch_loss_ud)
    end = time.time() - start
    length += end
    mean_so_far = length / (epoch+1)
    time_left = (mean_so_far * (epochs - epoch - 1))/60
    
    print(f"Epoch {epoch} of {epochs}:\t\t{end:.2f} seconds;\ttotal: {length:.2f};\tminutes left: {time_left:.2f}")
    print(f"Generator loss: {epoch_loss_g:.8f}, Discriminator loss: {epoch_loss_d:.5f}, Unrolled Discriminator loss: {epoch_loss_ud:.5f}")

Epoch 0 of 200:		43.42 seconds;	total: 43.42;	minutes left: 144.02
Generator loss: 2.18666410, Discriminator loss: 0.40314, Unrolled Discriminator loss: 0.30680
Epoch 1 of 200:		41.47 seconds;	total: 84.89;	minutes left: 140.07
Generator loss: 3.65209079, Discriminator loss: 0.09993, Unrolled Discriminator loss: 0.06859
Epoch 2 of 200:		41.23 seconds;	total: 126.12;	minutes left: 138.03
Generator loss: 7.85629129, Discriminator loss: 0.01929, Unrolled Discriminator loss: 0.00892
Epoch 3 of 200:		41.20 seconds;	total: 167.31;	minutes left: 136.64
Generator loss: 8.08483696, Discriminator loss: 0.01448, Unrolled Discriminator loss: 0.00757
Epoch 4 of 200:		41.31 seconds;	total: 208.62;	minutes left: 135.60
Generator loss: 5.77362394, Discriminator loss: 0.01621, Unrolled Discriminator loss: 0.00998
Epoch 5 of 200:		41.72 seconds;	total: 250.34;	minutes left: 134.91
Generator loss: 6.40051413, Discriminator loss: 0.01533, Unrolled Discriminator loss: 0.00879
Epoch 6 of 200:		41.18 seconds

KeyboardInterrupt: 

In [ ]:
discriminator

In [1]:
print('DONE TRAINING')
torch.save(generator.state_dict(), path + 'generator.pth')
path

DONE TRAINING


NameError: name 'torch' is not defined

In [22]:
imgs = [np.array(to_pil_image(img)) for img in images]
imageio.mimsave(path + 'generator_images.gif', imgs)

In [ ]:
plt.figure()
plt.plot(losses_g, label='Generator loss')
plt.plot(losses_d, label='Discriminator Loss')
plt.legend()
plt.savefig(path + 'loss.png')